# Load Test Seldon-Core on GCP Kubernetes



## Create Cluster

In [ ]:
!PROJECT=loadtesting
!ZONE=europe-west1-b
!gcloud beta container --project "${PROJECT}" clusters create "loadtest" \
    --zone "${ZONE}" \
    --username "admin" \
    --cluster-version "1.9.3-gke.0" \
    --machine-type "n1-standard-16" \
    --image-type "COS" \
    --disk-size "100" \
    --scopes "https://www.googleapis.com/auth/compute","https://www.googleapis.com/auth/devstorage.read_only","https://www.googleapis.com/auth/logging.write","https://www.googleapis.com/auth/monitoring","https://www.googleapis.com/auth/servicecontrol","https://www.googleapis.com/auth/service.management.readonly","https://www.googleapis.com/auth/trace.append" \
    --num-nodes "4" \
    --network "default" \
    --enable-cloud-logging \
    --enable-cloud-monitoring \
    --subnetwork "default"

## Install helm

In [1]:
!kubectl -n kube-system create sa tiller
!kubectl create clusterrolebinding tiller --clusterrole cluster-admin --serviceaccount=kube-system:tiller
!helm init --service-account tiller

serviceaccount "tiller" created
clusterrolebinding "tiller" created
$HELM_HOME has been configured at /home/clive/.helm.

Tiller (the Helm server-side component) has been installed into your Kubernetes Cluster.
Happy Helming!


## Start Seldon-Core CRD

In [2]:
!helm install ../helm-charts/seldon-core-crd --name seldon-core-crd

NAME:   seldon-core-crd
LAST DEPLOYED: Mon Mar  5 19:41:43 2018
NAMESPACE: default
STATUS: DEPLOYED

RESOURCES:
==> v1beta1/CustomResourceDefinition
NAME                                         AGE
seldondeployments.machinelearning.seldon.io  0s


NOTES:
NOTES: TODO




## Cordon off loadtest nodes

In [4]:
!kubectl get nodes

NAME                                      STATUS    ROLES     AGE       VERSION
gke-loadtest-default-pool-1bc67aef-5fz2   Ready     <none>    1m        v1.9.3-gke.0
gke-loadtest-default-pool-1bc67aef-5jpp   Ready     <none>    1m        v1.9.3-gke.0
gke-loadtest-default-pool-1bc67aef-rv06   Ready     <none>    2m        v1.9.3-gke.0
gke-loadtest-default-pool-1bc67aef-xqpp   Ready     <none>    2m        v1.9.3-gke.0


In [5]:
!kubectl cordon $(kubectl get nodes -o jsonpath='{.items[0].metadata.name}')
!kubectl cordon $(kubectl get nodes -o jsonpath='{.items[1].metadata.name}')
!kubectl cordon $(kubectl get nodes -o jsonpath='{.items[2].metadata.name}')

node "gke-loadtest-default-pool-1bc67aef-5fz2" cordoned
node "gke-loadtest-default-pool-1bc67aef-5jpp" cordoned
node "gke-loadtest-default-pool-1bc67aef-rv06" cordoned


In [6]:
!kubectl label nodes $(kubectl get nodes -o jsonpath='{.items[0].metadata.name}') role=locust
!kubectl label nodes $(kubectl get nodes -o jsonpath='{.items[1].metadata.name}') role=locust
!kubectl label nodes $(kubectl get nodes -o jsonpath='{.items[2].metadata.name}') role=locust

node "gke-loadtest-default-pool-1bc67aef-5fz2" labeled
node "gke-loadtest-default-pool-1bc67aef-5jpp" labeled
node "gke-loadtest-default-pool-1bc67aef-rv06" labeled


## Start seldon-core

In [7]:
!helm install ../helm-charts/seldon-core --name seldon-core \
        --set cluster_manager.rbac=true \
        --set apife.enabled=true \
        --set engine.image.tag=0.1.6_SNAPSHOT_loadtest \
        --set cluster_manager.image.tag=0.1.6_SNAPSHOT_loadtest
        

NAME:   seldon-core
LAST DEPLOYED: Mon Mar  5 19:42:28 2018
NAMESPACE: default
STATUS: DEPLOYED

RESOURCES:
==> v1beta1/Deployment
NAME                    DESIRED  CURRENT  UP-TO-DATE  AVAILABLE  AGE
seldon-apiserver        1        0        0           0          1s
seldon-cluster-manager  1        0        0           0          1s
redis                   1        1        1           0          1s

==> v1/Service
NAME              TYPE       CLUSTER-IP    EXTERNAL-IP  PORT(S)                        AGE
seldon-apiserver  NodePort   10.3.249.47   <none>       8080:30890/TCP,5000:30137/TCP  1s
redis             ClusterIP  10.3.240.187  <none>       6379/TCP                       1s

==> v1/ServiceAccount
NAME    SECRETS  AGE
seldon  1        1s

==> v1beta1/ClusterRoleBinding
NAME    AGE
seldon  1s

==> v1/Pod(related)
NAME                                     READY  STATUS             RESTARTS  AGE
seldon-cluster-manager-68c8c6b5bf-wvs6f  0/1    Pending            0         0s
redis-df

In [9]:
!kubectl get pods -o wide

NAME                                      READY     STATUS    RESTARTS   AGE       IP         NODE
redis-df886d999-c4bzl                     1/1       Running   0          1m        10.0.0.6   gke-loadtest-default-pool-1bc67aef-xqpp
seldon-apiserver-64ccd4c5f4-c25st         1/1       Running   0          1m        10.0.0.7   gke-loadtest-default-pool-1bc67aef-xqpp
seldon-cluster-manager-68c8c6b5bf-wvs6f   1/1       Running   0          1m        10.0.0.8   gke-loadtest-default-pool-1bc67aef-xqpp


## Create Stub Deployment

In [10]:
!kubectl apply -f resources/loadtest_simple_model.json

seldondeployment "seldon-core-loadtest" created


In [11]:
!kubectl get seldondeployments seldon-core-loadtest -o jsonpath='{.status}'

map[predictorStatus:[map[name:loadtest-loadtest replicas:1 replicasAvailable:1]]]

## Load test

In [97]:
!kubectl create clusterrolebinding default-admin --clusterrole=cluster-admin --serviceaccount=default:default

clusterrolebinding "default-admin" created


In [ ]:
!helm install ../helm-charts/seldon-core-analytics --name seldon-core-analytics \
    --set grafana_prom_admin_password=password \
    --set persistence.enabled=false

In [12]:
!kubectl uncordon $(kubectl get nodes -o jsonpath='{.items[0].metadata.name}')
!kubectl uncordon $(kubectl get nodes -o jsonpath='{.items[1].metadata.name}')
!kubectl uncordon $(kubectl get nodes -o jsonpath='{.items[2].metadata.name}')


node "gke-loadtest-default-pool-1bc67aef-5fz2" uncordoned
node "gke-loadtest-default-pool-1bc67aef-5jpp" uncordoned
node "gke-loadtest-default-pool-1bc67aef-rv06" uncordoned


In [19]:
!helm install ../helm-charts/seldon-core-loadtesting --name loadtest  \
    --set locust.host=loadtest:5001 \
    --set locust.script=predict_grpc_locust.py \
    --set oauth.enabled=false \
    --set oauth.key=oauth-key \
    --set oauth.secret=oauth-secret \
    --set locust.hatchRate=1 \
    --set locust.clients=512 \
    --set loadtest.sendFeedback=0 \
    --set locust.minWait=0 \
    --set locust.maxWait=0 \
    --set replicaCount=32 

NAME:   loadtest
LAST DEPLOYED: Mon Mar  5 19:57:30 2018
NAMESPACE: default
STATUS: DEPLOYED

RESOURCES:
==> v1/ReplicationController
NAME             DESIRED  CURRENT  READY  AGE
locust-slave-1   32       32       0      1s
locust-master-1  1        1        0      1s

==> v1/Service
NAME             TYPE      CLUSTER-IP    EXTERNAL-IP  PORT(S)                                       AGE
locust-master-1  NodePort  10.3.252.115  <none>       5557:30329/TCP,5558:30681/TCP,8089:30831/TCP  1s

==> v1/Pod(related)
NAME                   READY  STATUS             RESTARTS  AGE
locust-slave-1-28s2v   0/1    Pending            0         1s
locust-slave-1-2b6pp   0/1    ContainerCreating  0         1s
locust-slave-1-4g5ct   0/1    Pending            0         1s
locust-slave-1-4l5pg   0/1    Pending            0         1s
locust-slave-1-5jz2r   0/1    ContainerCreating  0         1s
locust-slave-1-6bgfr   0/1    Pending            0         1s
locust-slave-1-6mwjq   0/1    Pending            0 

In [17]:
!helm install ../helm-charts/seldon-core-loadtesting --name loadtest  \
    --set locust.host=http://loadtest:8000 \
    --set oauth.enabled=false \
    --set oauth.key=oauth-key \
    --set oauth.secret=oauth-secret \
    --set locust.hatchRate=1 \
    --set locust.clients=256 \
    --set loadtest.sendFeedback=0 \
    --set locust.minWait=0 \
    --set locust.maxWait=0 \
    --set replicaCount=32

NAME:   loadtest
LAST DEPLOYED: Mon Mar  5 19:54:27 2018
NAMESPACE: default
STATUS: DEPLOYED

RESOURCES:
==> v1/Pod(related)
NAME                   READY  STATUS             RESTARTS  AGE
locust-slave-1-5fzfk   0/1    Pending            0         0s
locust-slave-1-5qgc4   0/1    Pending            0         0s
locust-slave-1-6m52k   0/1    Pending            0         0s
locust-slave-1-7l7gt   0/1    ContainerCreating  0         0s
locust-slave-1-8d6p2   0/1    Pending            0         0s
locust-slave-1-9vgf8   0/1    Pending            0         0s
locust-slave-1-9wm9w   0/1    Pending            0         0s
locust-slave-1-d9vgs   0/1    Pending            0         0s
locust-slave-1-fz7vg   0/1    Pending            0         0s
locust-slave-1-g4w9m   0/1    Pending            0         0s
locust-slave-1-g898l   0/1    Pending            0         0s
locust-slave-1-gzxgt   0/1    Pending            0         0s
locust-slave-1-jsrdf   0/1    Pending            0         0s
locust

In [76]:
!kubectl cordon $(kubectl get nodes -o jsonpath='{.items[0].metadata.name}')
!kubectl cordon $(kubectl get nodes -o jsonpath='{.items[1].metadata.name}')
!kubectl cordon $(kubectl get nodes -o jsonpath='{.items[2].metadata.name}')
!kubectl cordon $(kubectl get nodes -o jsonpath='{.items[3].metadata.name}')

node "gke-loadtest-default-pool-b0c49eff-4l6d" cordoned
node "gke-loadtest-default-pool-b0c49eff-gn9z" cordoned
node "gke-loadtest-default-pool-b0c49eff-m3g5" cordoned
node "gke-loadtest-default-pool-b0c49eff-n6h3" cordoned


In [20]:
!helm delete loadtest --purge

release "loadtest" deleted


## Tear Down

In [ ]:
!kubectl delete -f resources/model.json

In [ ]:
!helm delete seldon-core --purge

In [ ]:
!helm delete seldon-core-crd --purge